In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# simulate the user profiles
users_data = {
    'user_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'skills': ['Python, Data Science', 'Java, Spring', 'JavaScript, React', 'Python, Machine Learning', 'Java, AWS'],
    'location': ['New York', 'San Francisco', 'London', 'Toronto', 'Berlin']
}

# Simulated job postings data
jobs_data = {
    'job_id': [101, 102, 103, 104, 105],
    'title': ['Data Scientist', 'Java Developer', 'Frontend Engineer', 'ML Engineer', 'AWS Architect'],
    'skills_required': ['Python, Machine Learning', 'Java, Spring', 'JavaScript, HTML', 'Python, Deep Learning', 'AWS, Cloud'],
    'location': ['New York', 'San Francisco', 'London', 'Toronto', 'Berlin']
}

# Simulated user-job interaction data
interactions_data = {
    'user_id': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
    'job_id': [101, 102, 103, 104, 105, 102, 101, 105, 103, 104],
    'applied': [1, 1, 0, 0, 1, 0, 1, 0, 1, 0]
}

In [ ]:
# convert these into data frames
users_df = pd.DataFrame(users_data)
jobs_df = pd.DataFrame(jobs_data)
interactions_df = pd.DataFrame(interactions_data)

In [ ]:
# Display the data
print("Users Data:")
print(users_df)
print("\nJobs Data:")
print(jobs_df)
print("\nInteractions Data:")
print(interactions_df)

Users Data:
   user_id     name                    skills       location
0        1    Alice      Python, Data Science       New York
1        2      Bob              Java, Spring  San Francisco
2        3  Charlie         JavaScript, React         London
3        4    David  Python, Machine Learning        Toronto
4        5      Eve                 Java, AWS         Berlin

Jobs Data:
   job_id              title           skills_required       location
0     101     Data Scientist  Python, Machine Learning       New York
1     102     Java Developer              Java, Spring  San Francisco
2     103  Frontend Engineer          JavaScript, HTML         London
3     104        ML Engineer     Python, Deep Learning        Toronto
4     105      AWS Architect                AWS, Cloud         Berlin

Interactions Data:
   user_id  job_id  applied
0        1     101        1
1        2     102        1
2        3     103        0
3        4     104        0
4        5     105        1
5 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Initialize CountVectorizer for skills
skill_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
user_skills_matrix = skill_vectorizer.fit_transform(users_df['skills']).toarray()
job_skills_matrix = skill_vectorizer.transform(jobs_df['skills_required']).toarray()

# Label encode locations
location_encoder = LabelEncoder()
users_df['location_encoded'] = location_encoder.fit_transform(users_df['location'])
jobs_df['location_encoded'] = location_encoder.transform(jobs_df['location'])

# Merge skills and locations into user/job profiles
user_profiles = np.hstack([user_skills_matrix, users_df['location_encoded'].values.reshape(-1, 1)])
job_profiles = np.hstack([job_skills_matrix, jobs_df['location_encoded'].values.reshape(-1, 1)])

print("User Profiles:")
print(user_profiles)

print("\nJob Profiles:")
print(job_profiles)


User Profiles:
[[0 1 0 0 0 1 0 0 2]
 [0 0 1 0 0 0 0 1 3]
 [0 0 0 1 0 0 1 0 1]
 [0 0 0 0 1 1 0 0 4]
 [1 0 1 0 0 0 0 0 0]]

Job Profiles:
[[0 0 0 0 1 1 0 0 2]
 [0 0 1 0 0 0 0 1 3]
 [0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 4]
 [1 0 0 0 0 0 0 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X = []
y = []

for _, row in interactions_df.iterrows():
  user_id = row['user_id']
  job_id = row['job_id']
  applied = row['applied']

  user_profile = user_profiles[user_id -1] # user profile vector
  job_profile = job_profiles[job_id-101] # job profile vector

  X.append(np.concatenate((user_profile, job_profile)))
  y.append(applied)

X = np.array(X)
y = np.array(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# simple feedforward neural network using TensorFlow to predict whether a user will apply for a job
from tensorflow import keras
from keras import layers

model = tf.keras.Sequential()

model.add(layers.Dense(128, activation = 'relu', input_shape = (X_train.shape[1],)))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(1, activation = 'sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# compiling the model
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# train the model
history = model.fit(X_train, y_train, epochs = 20,  batch_size = 4, validation_split = 0.1)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 210ms/step - accuracy: 0.6429 - loss: 0.6258 - val_accuracy: 0.0000e+00 - val_loss: 0.7761
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7262 - loss: 0.6006 - val_accuracy: 0.0000e+00 - val_loss: 0.8143
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5476 - loss: 0.6363 - val_accuracy: 0.0000e+00 - val_loss: 0.8544
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7262 - loss: 0.5127 - val_accuracy: 0.0000e+00 - val_loss: 0.9039
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5476 - loss: 0.6206 - val_accuracy: 0.0000e+00 - val_loss: 0.9503
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7262 - loss: 0.6422 - val_accuracy: 0.0000e+00 - val_loss: 1.0034
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8214 - loss: 0.5522 - val_accuracy: 0.0000e+00 - val_loss: 1.0525
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7262 - loss: 0.4440 - val_accurac

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {accuracy:.4f}")

1/1 - 0s - 33ms/step - accuracy: 0.0000e+00 - loss: 1.9156
Test accuracy: 0.0000


In [ ]:
def recommend_jobs_for_user(user_id, top_n=3):
    user_profile = user_profiles[user_id - 1]

    # Prepare input data for prediction
    job_recommendations = []
    for job_id in range(101, 106):
        job_profile = job_profiles[job_id - 101]
        input_features = np.concatenate((user_profile, job_profile)).reshape(1, -1)

        # Predict application likelihood
        prediction = model.predict(input_features)[0][0]
        job_recommendations.append((job_id, prediction))

    # Sort and recommend top N jobs
    job_recommendations.sort(key=lambda x: x[1], reverse=True)
    recommended_jobs = [job_id for job_id, _ in job_recommendations[:top_n]]

    print(f"Recommended Jobs for User {user_id}:")
    for job_id in recommended_jobs:
        job_title = jobs_df[jobs_df['job_id'] == job_id]['title'].values[0]
        print(f"Job ID: {job_id}, Title: {job_title}")

# recommendation for user 1
recommend_jobs_for_user(user_id=1, top_n=3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Recommended Jobs for User 1:
Job ID: 105, Title: AWS Architect
Job ID: 101, Title: Data Scientist
Job ID: 103, Title: Frontend Engineer


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between users
user_similarities = cosine_similarity(user_profiles)

# Function to recommend connections
def recommend_connections_for_user(user_id, top_n=3):
    user_idx = user_id - 1
    similarities = user_similarities[user_idx]

    # Sort and recommend top N connections (excluding self)
    similar_users = np.argsort(similarities)[::-1][1:top_n+1]

    print(f"Recommended Connections for User {user_id}:")
    for idx in similar_users:
        similar_user_id = idx + 1
        similar_user_name = users_df[users_df['user_id'] == similar_user_id]['name'].values[0]
        print(f"User ID: {similar_user_id}, Name: {similar_user_name}")

# Example recommendation for user 1
recommend_connections_for_user(user_id=1, top_n=3)

Recommended Connections for User 1:
User ID: 4, Name: David
User ID: 2, Name: Bob
User ID: 3, Name: Charlie


In [ ]:
def linkedin_recommendations(user_id, top_n_jobs=3, top_n_connections=3):
    print(f"Recommendations for User {user_id}:")
    recommend_jobs_for_user(user_id, top_n_jobs)
    recommend_connections_for_user(user_id, top_n_connections)

# Example recommendation for user 1
linkedin_recommendations(user_id=1, top_n_jobs=3, top_n_connections=3)

Recommendations for User 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Recommended Jobs for User 1:
Job ID: 105, Title: AWS Architect
Job ID: 101, Title: Data Scientist
Job ID: 103, Title: Frontend Engineer
Recommended Connections for User 1:
User ID: 4, Name: David
User ID: 2, Name: Bob
User ID: 3, Name: Charlie
